In [3]:
import pandas as pd
from pyspark.sql import *

In [6]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlc = SQLContext(sc)
posicion_mediciones = sqlc.read.csv("posicion_mediciones.csv").rdd
posicion_mediciones = posicion_mediciones.map(lambda x:(pd.to_numeric(x[0],"coerce"),x[1],\
                                    x[2],pd.to_numeric(x[3],"coerce"),pd.to_numeric(x[4],"coerce"))).cache()
mediciones = sqlc.read.csv("mediciones.csv").rdd
mediciones = mediciones.map(lambda x:(pd.to_datetime(x[0],"coerce"),pd.to_numeric(x[1],"coerce"),\
                             pd.to_numeric(x[2],"coerce"),pd.to_numeric(x[3],"coerce"),\
                             pd.to_numeric(x[4],"coerce"),x[5],pd.to_numeric(x[6],"coerce"))).cache()

In [10]:
#posicion_mediciones: (ID_BASE,NOMBRE,PCIA, CIUDAD, LAT, LON)
#mediciones: (TIMESTAMP,ID_BASE,TEMPERATURA,HUMEDAD,PRESIÓN,DIRECCIÓN VIENTO, VELOCIDAD VIENTO)

#COMPLETAR, HACERLO BIEN

def hay_dif_30(lista):
    

mediciones_bs_as = posicion_mediciones.filter(lambda x:x[2]=="Buenos Aires").map(lambda x:(x[0],x[1]))#(id,nombre)
mediciones_2018 = mediciones.filter(lambda x:x[0].year==2018).\
        map(lambda x:((x[0].month,x[1]),(x[2],1)))#((mes,id_base),(temperatura,1))

#falta el join con las de bs as

mediciones_2018_promedio_mes = mediciones_2018.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).\
        map(lambda x:((x[0][0],x[0][1]),x[1][0]/x[1][1]))#((mes,id base),promedio temp)
promedio_base_por_mes = mediciones_2018_promedio_mes.map(lambda x:(x[0][1],[(x[0][0],x[1])])).reduceByKey(lambda x,y:x+y).\
        map(lambda x:(x[0],sorted(x[1])))#(id_base,lista de (mes,promedio temp))

[(1,
  [(1, 21.6),
   (2, 22.0),
   (3, 15.555555555555555),
   (4, 24.5),
   (5, 11.333333333333334),
   (6, 18.428571428571427),
   (7, 28.125),
   (8, 21.5),
   (9, 17.428571428571427),
   (10, 19.90909090909091),
   (11, 20.0),
   (12, 13.285714285714286)]),
 (2,
  [(1, 15.636363636363637),
   (2, 21.7),
   (3, 23.9),
   (4, 19.363636363636363),
   (5, 7.5),
   (6, 16.6),
   (7, 14.7),
   (8, 23.571428571428573),
   (9, 18.333333333333332),
   (10, 20.1),
   (11, 24.555555555555557),
   (12, 25.4)]),
 (5,
  [(1, 19.642857142857142),
   (2, 21.733333333333334),
   (3, 13.166666666666666),
   (4, 23.77777777777778),
   (5, 26.0),
   (6, 27.5),
   (7, 16.375),
   (8, 25.46153846153846),
   (9, 29.4),
   (10, 14.454545454545455),
   (11, 14.375),
   (12, 15.666666666666666)]),
 (3,
  [(1, 24.0),
   (2, 28.0),
   (3, 14.75),
   (4, 22.571428571428573),
   (5, 17.1),
   (6, 22.0),
   (7, 20.6),
   (8, 12.375),
   (9, 15.9),
   (10, 23.857142857142858),
   (11, 23.571428571428573),
   (12